Multimodal entailment involves analysis of text, images, audio, and video data sources to determine if a piece of information contradict another or whether a given piece of information implies the other. This is applied is social media content moderation where platform operators audits and moderate content.

In [ ]:
!pip install -q tensorflow_text

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import math
from skimage.io import imread
from skimage.transform import resize
from PIL import Image
import os
import matplotlib.pyplot as plt

os.environ['KERAS_BACKEND'] = 'jax'

import keras
import keras_hub
from keras.utils import PyDataset

In [ ]:
# Define a label map
label_map = {
    'contradiction': 0,
    'entailment': 1,
    'neutral': 2
}

In [ ]:
# Collect dataset
image_base_path = keras.utils.get_file(
    "tweet_images",
    "https://github.com/sayakpaul/Multimodal-Entailment-Baseline/releases/download/v1.0.0/tweet_images.tar.gz",
    untar=True,
)

# Read dataset and apply preprocessing to the first 1k samples
df = pd.read_csv(
    "https://github.com/sayakpaul/Multimodal-Entailment-Baseline/raw/main/csvs/tweets.csv"
).iloc[0:1000]

df.sample(10)

In [ ]:
# We formulate the entailment task as - given pairs of {text_1, image_1} and {text_2, image_2}, predict (contradict, entail, or neutral)

images_1_paths = []
images_2_paths = []

for i in range(len(df)):
    current_row = df.iloc[i]
    id_1 = current_row["id_1"]
    id_2 = current_row["id_2"]
    extension_1 = current_row["image_1"].split(".")[-1]
    extension_2 = current_row["image_2"].split(".")[-1]
    image_1_path = os.path.join(image_base_path, f"{id_1}.{extension_1}")
    image_2_path = os.path.join(image_base_path, f"{id_2}.{extension_2}")
    images_1_paths.append(image_1_path)
    images_2_paths.append(image_2_path)

df["image_1_path"] = images_1_paths
df["image_2_path"] = images_2_paths

# Add label column
df['label_idx'] = df['label'].apply(lambda x: label_map[x])

In [ ]:
# Visualize sample dataset
def visualize(idx):
    current_row = df.iloc[idx]
    image_1 = plt.imread(current_row["image_1_path"])
    image_2 = plt.imread(current_row["image_2_path"])
    text_1 = current_row["text_1"]
    text_2 = current_row["text_2"]
    label = current_row["label"]

    plt.subplot(1, 2, 1)
    plt.imshow(image_1)
    plt.axis("off")
    plt.title(f"Image 1: {text_1}")

    plt.subplot(1, 2, 2)
    plt.imshow(image_2)
    plt.axis("off")
    plt.title(f"Image 2: {text_2}")
    plt.show()
    print(f"Label: {label}")

random_idx = random.choice(range(len(df)))
visualize(random_idx)

random_idx = random.choice(range(len(df)))
visualize(random_idx)


In [ ]:
# Train-Test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'].values, random_state=42)

# validation set
train_df, val_df = train_test_split(train_df, test_size=0.05, stratify=train_df['label'].values, random_state=42)

print(f"Total train examples: {len(train_df)}")
print(f"Total val examples: {len(val_df)}")
print(f"Total test examples: {len(test_df)}"

In [ ]:
# Data input pipeline
text_preprocessor = keras_hub.models.BertTextClassifierPreprocessor.from_preset("bert_base_en_uncased", sequence_lengh=128,
)

In [ ]:
# Run the preprocessor on a sample input
idx = random.choice(range(len(train_df)))
sample_input = train_df.iloc[idx]
sample_text_1 = sample_input["text_1"]
sample_text_2 = sample_input["text_2"]

print(f"Text 1: {sample_text_1}")
print(f"Text 2: {sample_text_2}")

processed_text = text_preprocessor([sample_text_1, sample_text_2])
print(processed_text)

print("Keys            : ", list(processed_text.keys()))
print("Shape Token Ids : ", processed_text['token_ids'].shape)
print("Token Ids       : ", processed_text['token_ids'][0, :16])
print("Shape Padding Masks     : ", processed_text['padding_mask'].shape)
print("Padding Masks     : ", processed_text['padding_mask'][0, :16])
print("Shape Segment Ids : ", processed_text['segment_ids'].shape)
print("Segment Ids       : ", processed_text['segment_ids'][0, :16])

In [ ]:
# Create tf.data.Dataset objects from dataframes
def dataframe_to_dataset(dataframe):
    columns = ['image_1_path', 'image_2_path', 'text_1', 'text_2', 'label_idx']
    dataset = UnifiedPyDataset(dataframe, batch_size=32, workers=4)
    return dataset

# Preprocessing utiliteis
bert_input_features = ['padding_mask', 'segment_ids', 'token_ids']
def preprocess_text(text_1, text_2):
    output = text_preprocessor([text_1, text_2])
    return {feature: keras.ops.reshape(output[feature], [-1]) for feature in bert_input_features}

In [ ]:
class UnifiedPyDataset(PyDataset):
    """A Keras-compatible dataset that processes a DataFrame for TensorFlow, JAX, and PyTorch."""

    def __init__(
        self,
        df,
        batch_size=32,
        workers=4,
        use_multiprocessing=False,
        max_queue_size=10,
        **kwargs,
    ):
        """
        Args:
            df: pandas DataFrame with data
            batch_size: Batch size for dataset
            workers: Number of workers to use for parallel loading (Keras)
            use_multiprocessing: Whether to use multiprocessing
            max_queue_size: Maximum size of the data queue for parallel loading
        """
        super().__init__(**kwargs)
        self.dataframe = df
        columns = ["image_1_path", "image_2_path", "text_1", "text_2"]

        # image files
        self.image_x_1 = self.dataframe["image_1_path"]
        self.image_x_2 = self.dataframe["image_1_path"]
        self.image_y = self.dataframe["label_idx"]

        # text files
        self.text_x_1 = self.dataframe["text_1"]
        self.text_x_2 = self.dataframe["text_2"]
        self.text_y = self.dataframe["label_idx"]

        # general
        self.batch_size = batch_size
        self.workers = workers
        self.use_multiprocessing = use_multiprocessing
        self.max_queue_size = max_queue_size

    def __getitem__(self, index):
        """
        Fetches a batch of data from the dataset at the given index.
        """

        # Return x, y for batch idx.
        low = index * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.

        high_image_1 = min(low + self.batch_size, len(self.image_x_1))
        high_image_2 = min(low + self.batch_size, len(self.image_x_2))

        high_text_1 = min(low + self.batch_size, len(self.text_x_1))
        high_text_2 = min(low + self.batch_size, len(self.text_x_1))

        # images files
        batch_image_x_1 = self.image_x_1[low:high_image_1]
        batch_image_y_1 = self.image_y[low:high_image_1]

        batch_image_x_2 = self.image_x_2[low:high_image_2]
        batch_image_y_2 = self.image_y[low:high_image_2]

        # text files
        batch_text_x_1 = self.text_x_1[low:high_text_1]
        batch_text_y_1 = self.text_y[low:high_text_1]

        batch_text_x_2 = self.text_x_2[low:high_text_2]
        batch_text_y_2 = self.text_y[low:high_text_2]

        # image number 1 inputs
        image_1 = [
            resize(imread(file_name), (128, 128)) for file_name in batch_image_x_1
        ]
        image_1 = [
            (  # exeperienced some shapes which were different from others.
                np.array(Image.fromarray((img.astype(np.uint8))).convert("RGB"))
                if img.shape[2] == 4
                else img
            )
            for img in image_1
        ]
        image_1 = np.array(image_1)

        # Both text inputs to the model, return a dict for inputs to BertBackbone
        text = {
            key: np.array(
                [
                    d[key]
                    for d in [
                        preprocess_text(file_path1, file_path2)
                        for file_path1, file_path2 in zip(
                            batch_text_x_1, batch_text_x_2
                        )
                    ]
                ]
            )
            for key in ["padding_mask", "token_ids", "segment_ids"]
        }

        # Image number 2 model inputs
        image_2 = [
            resize(imread(file_name), (128, 128)) for file_name in batch_image_x_2
        ]
        image_2 = [
            (  # exeperienced some shapes which were different from others
                np.array(Image.fromarray((img.astype(np.uint8))).convert("RGB"))
                if img.shape[2] == 4
                else img
            )
            for img in image_2
        ]
        # Stack the list comprehension to an nd.array
        image_2 = np.array(image_2)

        return (
            {
                "image_1": image_1,
                "image_2": image_2,
                "padding_mask": text["padding_mask"],
                "segment_ids": text["segment_ids"],
                "token_ids": text["token_ids"],
            },
            # Target lables
            np.array(batch_image_y_1),
        )

    def __len__(self):
        """
        Returns the number of batches in the dataset.
        """
        return math.ceil(len(self.dataframe) / self.batch_size)